# Cortes en Volumen para simulaciones de random direction y random vertex

In [1]:
import uproot
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm

import seaborn as sn

import torch

# 1 - Extracting Data

In [2]:
file = uproot.open("/snoplus simulations/electron bulk/random point/random dir/Low Statistic/simu_Analysis_elec_10MeV.root")
file.keys()

['T;6', 'T;5', 'pmt;2', 'pmt;1']

In [3]:
data1 = file['T;6']
data1.keys()

['evtid',
 'mcID',
 'energy',
 'mc_position',
 'mc_momentum',
 'position',
 'momentum',
 'hit_pmtid',
 'hit_pmttime',
 'hit_residual',
 'hit_type']

In [4]:
pmt_info = file['pmt;1']
pmt_info.keys()

['pmt_id', 'pmt_pos_xyz', 'pmt_pos_sph', 'pmt_type']

## 1.1 - EV info

In [5]:
evtid = np.array(data1['evtid'])
mcID = np.array(data1['mcID'])
mc_position = np.array(data1['mc_position'])  # -> Position Generation
mc_momentum = np.array(data1['mc_momentum'])  # -> IMPORTANT: Direction of simulated event
position = np.array(data1['position'])        
hit_pmtid = np.array(data1['hit_pmtid'])
hit_pmttime = np.array(data1['hit_pmttime'])
time_residual = np.array(data1['hit_residual'])
hit_type = np.array(data1['hit_type'])

## 1.2 - PMT info

In [6]:
pmt_id = np.array(pmt_info['pmt_id'])
pmt_pos_xyz = np.array(pmt_info['pmt_pos_xyz'])
pmt_pos_sph = np.array(pmt_info['pmt_pos_sph'])
pmt_type = np.array(pmt_info['pmt_type'])

# 2 - Extract Valid Data

## 2.1 - Extract  valid PMT id

In [7]:
## Numpy proofs to validate pytorch tools

valid_pmt_id = np.where(pmt_type == 1)[0]   # PMT valid ID match with index of where pmt_type=1

valid_id_info_ev_i = np.where(np.in1d(hit_pmtid, valid_pmt_id))[0]
valid_id_info_ev_i

array([       0,        1,        2, ..., 11158432, 11158433, 11158434],
      dtype=int64)

## 2.2 - Extract valid info of EV


In [8]:
# CONCATENATE DATA IN NUMPY ARRAY IS FASTER!
valid_pmt_id = np.where(pmt_type == 1)[0]   #pmt valid ID match with index of where pmt_type=1
valid_pmt_id_ev = []

# With valid index data extract valid data

data_portion = 1   # percentage of data
lim_data = round(len(valid_id_info_ev_i)*data_portion)

multi_time_residual = []
multi_pmt_id_hit = []
multi_mc_position = []
multi_mc_momentum = []   
multi_xyz_hit = []

for valid_i in valid_id_info_ev_i[0:lim_data]:
    multi_time_residual.append(time_residual[valid_i])
    multi_mc_position.append(mc_position[valid_i])
    multi_mc_momentum.append(mc_momentum[valid_i])
    multi_pmt_id_hit.append(hit_pmtid[valid_i])
    
multi_time_residual = np.array(multi_time_residual)   #Has to be an array to use numpoy tools bellow!

#extract xyz_hit from pmt_id_hit
for id_i in multi_pmt_id_hit:
    multi_xyz_hit.append(pmt_pos_xyz[id_i])

# 3 - Cortes en Volumen

Usar radio de mc_position para seleccionar eventos

## 3.1 - Función para calcular radio de mc_position

In [9]:
def magnitude(vector): 
    norm = []
    for vec in vector:
    	norm.append(np.linalg.norm(vec))

    return np.array(norm)

In [10]:
mc_position_norm = magnitude(multi_mc_position)

In [11]:
mc_position_norm

array([2706.35218464, 2706.35218464, 2706.35218464, ..., 5946.13670468,
       5946.13670468, 5946.13670468])

## 3.2 - Corte en Radio

In [84]:
#Radi selection
max_radi = 5500.0
radi_condition = (mc_position_norm < max_radi)
index_cut_radi = np.where(radi_condition)[0]

#extract variables

multi_time_res_radi_cut = []
multi_mc_momentum_radi_cut = []
multi_pmt_id_hit_radi_cut = []
multi_xyz_hit_radi_cut = []

for j_dx in index_cut_radi:
    multi_time_res_radi_cut.append(multi_time_residual[j_dx])
    multi_mc_momentum_radi_cut.append(multi_mc_momentum[j_dx])
    multi_pmt_id_hit_radi_cut.append(multi_pmt_id_hit[j_dx])   
multi_time_res_radi_cut = np.array(multi_time_res_radi_cut)

for id_i in multi_pmt_id_hit_radi_cut:
    multi_xyz_hit_radi_cut.append(pmt_pos_xyz[id_i])

## 3.3 - Calculo de Observable $cos(α)$

In [82]:
multi_cos_angle = []
N = np.shape(multi_xyz_hit_radi_cut)[0]

for k in range(N):
    cos_val = np.dot(multi_xyz_hit_radi_cut[k], multi_mc_momentum_radi_cut[k])/(np.linalg.norm(multi_mc_momentum_radi_cut[k])*np.linalg.norm(multi_xyz_hit_radi_cut[k]))
    multi_cos_angle.append(cos_val)

# 4 - Plots

## 4.1 - Time residual vs. $cos(α)$

### 4.1.1 - cut time residual

In [98]:
up_cut = 20
inf_cut = -5

multi_time_res_cut = []
multi_cos_angle_cut = []

for i in np.where((multi_time_res_radi_cut > inf_cut) & (multi_time_res_radi_cut < up_cut))[0]:

    multi_time_res_cut.append(multi_time_res_radi_cut[i])
    multi_cos_angle_cut.append(multi_cos_angle[i])

In [99]:
%matplotlib
# 2d Histogram
bins = 40

title = 'cos(α) vs. time residual - Multiple evID - 10MeV - random (dir,vtx) - Vol. Cut'

plt.figure(figsize=(12, 10))
sn.set_style(rc = {'axes.facecolor': 'black'})
sn.histplot(x = multi_cos_angle_cut, y = multi_time_res_cut, bins = [bins, bins], stat='count', cbar = 'True', cmap = cm.nipy_spectral)
plt.ylabel('time residual')
plt.xlabel('cos(α)')
plt.title(title)

#equal acis ration
#ax = plt.gca()
#ax.set_aspect('equal', adjustable='box')
plt.show()
#plt.savefig('figs/Low Stat/' + title +'.png', format = 'png')

Using matplotlib backend: TkAgg


### 4.1.2 - Normalozation

#### 4.1.2.1 - Normalización por Nhits

In [96]:
#Transforma Data into an Image Matrix
H, xedges, yedges = np.histogram2d(x = multi_cos_angle_cut, y = multi_time_res_cut, bins = [bins, bins])

#Now recover the data axis representation of time residual vs cos(alpha)
left = np.min(multi_time_res_cut)
right = np.max(multi_time_res_cut)
bottom = np.min(multi_cos_angle_cut)
top = np.max(multi_cos_angle_cut)

#Titles
title = 'cos(α) vs. time residual- Multiple evID - 10 MeV - random (dir,vtx) - Vol. Cut'
ylabel = 'cos(α)'
xlabel = 'time residual'

plt.imshow(H, extent = [left, right, top, bottom], aspect="auto" ) #---> Here is our image as a Matrix Pixel
plt.title(title)
plt.ylabel(ylabel)
plt.xlabel(xlabel)
plt.colorbar()

In [100]:
# sum of hits in each bin = sum over each column element of the matrix.
sum_hit = np.sum(H,axis=0)  

H_normalized1 = H / sum_hit

N1title = 'Normalization by NHits - Multiple evID - 10 MeV - random (dir,vtx) - Vol. Cut'

plt.figure(figsize=(12, 10))
plt.imshow(H_normalized1, extent = [left, right, top, bottom], aspect="auto" )
plt.title(N1title)
plt.ylabel(ylabel)
plt.xlabel(xlabel)
plt.colorbar()
plt.savefig('figs/Low Stat/' + N1title +'.png', format = 'png')